In [1]:
import numpy as np
from numpy import genfromtxt
import tensorflow as tf

In [2]:
train = genfromtxt('./train_data.csv', delimiter=',')
label = genfromtxt('./data_label.csv', delimiter=',')

In [3]:
print("Train dataset\t:", train.shape)
print("Train labels\t:", label.shape)

('Train dataset\t:', (1259, 20))
('Train labels\t:', (1259,))


In [4]:
print label[12]

1.0


In [5]:
label = (np.arange(2) == label[:, None]).astype(np.float32)
print(label)
print(label.shape)

[[ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 ..., 
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]]
(1259, 2)


In [6]:
def init_weights(shape):
    return tf.Variable(tf.random_normal(shape, stddev=0.01))

In [7]:
def model(X, w_h, w_h2, w_o, p_keep_input, p_keep_hidden): # this network is the same as the previous one except with an extra hidden layer + dropout
    X = tf.nn.dropout(X, p_keep_input)
    h = tf.nn.relu(tf.matmul(X, w_h))

    h = tf.nn.dropout(h, p_keep_hidden)
    h2 = tf.nn.relu(tf.matmul(h, w_h2))

    h2 = tf.nn.dropout(h2, p_keep_hidden)
    
    return tf.matmul(h2, w_o)

In [8]:
X = tf.placeholder("float", [None, 20])
Y = tf.placeholder("float", [None, 2])

In [9]:
w_h = init_weights([20, 128])
w_h2 = init_weights([128, 32])
w_o = init_weights([32, 2])

In [10]:
p_keep_input = tf.placeholder("float")
p_keep_hidden = tf.placeholder("float")
py_x = model(X, w_h, w_h2, w_o, p_keep_input, p_keep_hidden)

In [11]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=py_x, labels=Y))
train_op = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cost)
predict_op = tf.argmax(py_x, 1)

In [12]:
trX, trY, teX, teY = train[:1000], label[:1000], train[1000:], label[1000:]

In [13]:
with tf.Session() as sess:
    # you need to initialize all variables
    tf.global_variables_initializer().run()
    
    for i in range(20):
        for start, end in zip(range(0, len(trX), 128), range(128, len(trX)+1, 128)):
            sess.run(train_op, feed_dict={X: trX[start:end], Y: trY[start:end],
                                          p_keep_input: 0.8, p_keep_hidden: 0.5})
        print(i, np.mean(np.argmax(teY, axis=1) ==
                         sess.run(predict_op, feed_dict={X: teX, 
                                                         p_keep_input: 1.0,
                                                            p_keep_hidden: 1.0})))

(0, 0.18532818532818532)
(1, 0.18532818532818532)
(2, 0.18532818532818532)
(3, 0.18532818532818532)
(4, 0.18532818532818532)
(5, 0.18532818532818532)
(6, 0.18532818532818532)
(7, 0.18532818532818532)
(8, 0.18532818532818532)
(9, 0.27413127413127414)
(10, 0.18532818532818532)
(11, 0.81467181467181471)
(12, 0.81467181467181471)
(13, 0.81467181467181471)
(14, 0.81467181467181471)
(15, 0.81467181467181471)
(16, 0.81467181467181471)
(17, 0.81467181467181471)
(18, 0.81467181467181471)
(19, 0.81467181467181471)
